# IMPORT LIBARY

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import tkinter as tk 

from tkinter import ttk,messagebox  

In [ ]:
x = pd.read_csv('datasets/smartphone_battery_features.csv')
y = pd.read_csv('datasets/smartphone_battery_targets.csv')
df = pd.concat([x, y], axis=1)

df.head()

In [ ]:
# Cek Tipe Data Pada Setiap Kolom
df.info()

In [ ]:
dimension = df.shape 
print(f"Dimensi Dari Dataset : {dimension}")

MENETUKAN NAMA FITUR

In [ ]:
df.columns

In [ ]:
df.describe()

MENANGANI DATA YANG BERMASALAH

In [ ]:
count_duplicated = df.duplicated().sum()
print(f'Jumlah Data Duplikat : {count_duplicated}')

count_isna = df.isna().sum().sum()
print(f"Jumlah Data Kosong Pada Dataset : {count_isna}")

PENANGANAN APABILA TERDAPAT MISSING VALUE

In [ ]:
if count_isna > 0:
    for x in df.columns:
        if df[x].dtype == 'Object':
            df[x] = df[x].fillna().mode()[0]
        else:
            df[x] = df[x].fillna().mean()

else :
    print("Tidak Ada Data Yang Perlu Ditangani")

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.drop(columns=['Device_ID'])

In [ ]:
df.head(10)

MENENTUKAN LABEL X DAN Y

In [ ]:
# background_app_usage_level 
# signal_strengt_avg 
# recomended_action

In [ ]:
map_background = {'Low':1 , 'Medium':2, 'High':3}
df['background_app_usage_level'] = df['background_app_usage_level'].replace('Low', 1)


In [ ]:
df.head(10)

In [ ]:
x = df[[]]

In [ ]:
for i in df.columns:
    sns.boxplot(df[i])
    plt.show()